<a href="https://colab.research.google.com/github/aishwaryasnair0-create/OSCC/blob/main/FInal%20OSCC_Risk_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
base_dir = '/content/drive/MyDrive/oscc_mirna'
!ls "$base_dir"


data	  environment.yml  mapping.zip	raw.zip  results.zip  scripts.zip
data.zip  mapping	   raw		results  scripts


In [4]:
# Basic imports (use Colab's default versions)
import os, random
import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
import seaborn as sns

# Reproducibility
RANDOM_SEED = 42
os.environ["PYTHONHASHSEED"] = str(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

print("Imports OK")
print("numpy:", np.__version__)
print("pandas:", pd.__version__)


Imports OK
numpy: 2.0.2
pandas: 2.2.2


In [5]:
import os

DATA_DIR = os.path.join(base_dir, 'data')
print("DATA_DIR =", DATA_DIR)

# List files inside the data folder
!ls "$DATA_DIR"


DATA_DIR = /content/drive/MyDrive/oscc_mirna/data
GSE113956_expr_miRNA_zscored.tsv
GSE113956_expr_probelevel.tsv
GSE113956_sample_metadata_fixed.tsv
GSE113956_sample_metadata_raw.tsv
GSE168227_expr_miRNA_zscored.tsv
GSE168227_expr_probelevel.tsv
GSE168227_sample_metadata_fixed.tsv
GSE168227_sample_metadata_raw.tsv
GSE241289_expr_miRNA_zscored.tsv
GSE241289_expr_probelevel.tsv
GSE241289_sample_metadata_fixed.tsv
GSE241289_sample_metadata_raw.tsv
GSE28100_expr_miRNA_zscored.tsv
GSE28100_expr_probelevel.tsv
GSE28100_sample_metadata_fixed.tsv
GSE28100_sample_metadata_raw.tsv
GSE31277-GPL9770_expr_miRNA_zscored.tsv
GSE31277-GPL9770_expr_probelevel.tsv
GSE31277-GPL9770_sample_metadata_fixed.tsv
GSE31277-GPL9770_sample_metadata_raw.tsv
GSE45238_expr_miRNA_zscored.tsv
GSE45238_expr_probelevel.tsv
GSE45238_expr_zscored.tsv
GSE45238_ILMN_to_miRNA_mapping.tsv
GSE45238_sample_metadata_fixed.tsv
GSE45238_sample_metadata_raw.tsv
GSE98463_expr_miRNA_zscored.tsv
GSE98463_expr_probelevel.tsv
GSE98463_s

In [6]:
# Test loading ONE GEO dataset: GSE28100
gse_id = 'GSE28100'

expr_path = os.path.join(DATA_DIR, f'{gse_id}_expr_miRNA_zscored.tsv')
meta_path = os.path.join(DATA_DIR, f'{gse_id}_sample_metadata_fixed.tsv')

print("Expression file:", expr_path)
print("Metadata file  :", meta_path)

expr = pd.read_csv(expr_path, sep='\t')
meta = pd.read_csv(meta_path, sep='\t')

print("\nExpression shape:", expr.shape)
print("Metadata shape  :", meta.shape)

print("\nExpression columns (first 10):")
print(expr.columns[:10])

print("\nMetadata columns (first 10):")
print(meta.columns[:10])


Expression file: /content/drive/MyDrive/oscc_mirna/data/GSE28100_expr_miRNA_zscored.tsv
Metadata file  : /content/drive/MyDrive/oscc_mirna/data/GSE28100_sample_metadata_fixed.tsv

Expression shape: (20, 940)
Metadata shape  : (20, 38)

Expression columns (first 10):
Index(['SampleID', 'bkv-miR-B1-3p', 'bkv-miR-B1-5p', 'ebv-miR-BART1-3p',
       'ebv-miR-BART1-5p', 'ebv-miR-BART10', 'ebv-miR-BART10*',
       'ebv-miR-BART11-3p', 'ebv-miR-BART11-5p', 'ebv-miR-BART12'],
      dtype='object')

Metadata columns (first 10):
Index(['SampleID', 'status', 'submission_date', 'last_update_date', 'type',
       'channel_count', 'source_name_ch1', 'organism_ch1',
       'characteristics_ch1', 'treatment_protocol_ch1'],
      dtype='object')


In [7]:
# Merge expression + metadata for GSE28100 and inspect label-like columns

# 1) Safety check for SampleID column
print("Expression columns include SampleID:", 'SampleID' in expr.columns)
print("Metadata columns include SampleID  :", 'SampleID' in meta.columns)

if 'SampleID' not in expr.columns or 'SampleID' not in meta.columns:
    print("\nSampleID is missing in one of the tables. Columns are:")
    print("expr columns:", expr.columns.tolist())
    print("meta columns:", meta.columns.tolist())
else:
    merged = expr.merge(meta, on='SampleID', how='inner')
    print("\nMerged shape:", merged.shape)

    # 2) Find candidate label / group columns
    candidate_cols = [
        c for c in merged.columns
        if any(k in c.lower() for k in ['group', 'tumour', 'tumor', 'normal', 'case', 'control'])
    ]

    print("\nCandidate label/group columns:")
    for c in candidate_cols:
        print(" -", c)

    # 3) Show value counts for each candidate
    for c in candidate_cols:
        print(f"\nValue counts for {c}:")
        print(merged[c].value_counts(dropna=False))


Expression columns include SampleID: True
Metadata columns include SampleID  : True

Merged shape: (20, 977)

Candidate label/group columns:
 - SampleType_TumourNormal
 - Group_OSCC_vs_nonOSCC
 - Group_3class_global

Value counts for SampleType_TumourNormal:
SampleType_TumourNormal
Tumour    17
Normal     3
Name: count, dtype: int64

Value counts for Group_OSCC_vs_nonOSCC:
Group_OSCC_vs_nonOSCC
1    17
0     3
Name: count, dtype: int64

Value counts for Group_3class_global:
Group_3class_global
1    17
0     3
Name: count, dtype: int64


In [8]:
# Define the diagnostic label column for OSCC vs normal
DIAG_LABEL_COL = 'Group_OSCC_vs_nonOSCC'

# Re-create merged just to be sure we are using the latest expr/meta
merged_28100 = expr.merge(meta, on='SampleID', how='inner')
print("Merged GSE28100 shape:", merged_28100.shape)

# Check label distribution
print("\nValue counts for", DIAG_LABEL_COL, "in GSE28100:")
print(merged_28100[DIAG_LABEL_COL].value_counts(dropna=False))

print("\nInterpretation: 1 = OSCC (tumour), 0 = normal")


Merged GSE28100 shape: (20, 977)

Value counts for Group_OSCC_vs_nonOSCC in GSE28100:
Group_OSCC_vs_nonOSCC
1    17
0     3
Name: count, dtype: int64

Interpretation: 1 = OSCC (tumour), 0 = normal


In [13]:
DIAG_LABEL_COL = 'Group_OSCC_vs_nonOSCC'  # 1 = OSCC, 0 = normal

geo_datasets = [
    'GSE28100',
    'GSE45238',
    'GSE168227',
    'GSE113956',
    'GSE241289',
    'GSE31277-GPL9770',
    'GSE98463',
]

def load_and_prepare_geo(gse_id: str) -> pd.DataFrame:
    expr_path = os.path.join(DATA_DIR, f'{gse_id}_expr_miRNA_zscored.tsv')
    meta_path = os.path.join(DATA_DIR, f'{gse_id}_sample_metadata_fixed.tsv')

    print(f'\n=== {gse_id} ===')
    print("Expression:", expr_path)
    print("Metadata  :", meta_path)

    expr = pd.read_csv(expr_path, sep='\t')
    meta = pd.read_csv(meta_path, sep='\t')

    # ---- Fix for datasets where expr has no 'SampleID' (e.g. GSE45238) ----
    if 'SampleID' not in expr.columns:
        first_col = expr.columns[0]
        print(f"  No 'SampleID' in expr; renaming first column '{first_col}' -> 'SampleID'")
        expr = expr.rename(columns={first_col: 'SampleID'})

    if 'SampleID' not in meta.columns:
        raise ValueError(f"{gse_id}: 'SampleID' missing in metadata")

    merged = expr.merge(meta, on='SampleID', how='inner')
    print("  Merged shape:", merged.shape)

    if DIAG_LABEL_COL not in merged.columns:
        raise ValueError(f"{gse_id}: diagnostic label column '{DIAG_LABEL_COL}' not found")

    df = merged.dropna(subset=[DIAG_LABEL_COL]).copy()
    df[DIAG_LABEL_COL] = df[DIAG_LABEL_COL].astype(int)
    print("  Label value counts:", df[DIAG_LABEL_COL].value_counts(dropna=False).to_dict())
    return df

# Reload all GEO datasets with the fixed loader
geo_tables = {}
for gse in geo_datasets:
    geo_tables[gse] = load_and_prepare_geo(gse)



=== GSE28100 ===
Expression: /content/drive/MyDrive/oscc_mirna/data/GSE28100_expr_miRNA_zscored.tsv
Metadata  : /content/drive/MyDrive/oscc_mirna/data/GSE28100_sample_metadata_fixed.tsv
  Merged shape: (20, 977)
  Label value counts: {1: 17, 0: 3}

=== GSE45238 ===
Expression: /content/drive/MyDrive/oscc_mirna/data/GSE45238_expr_miRNA_zscored.tsv
Metadata  : /content/drive/MyDrive/oscc_mirna/data/GSE45238_sample_metadata_fixed.tsv
  No 'SampleID' in expr; renaming first column 'Unnamed: 0' -> 'SampleID'
  Merged shape: (80, 910)
  Label value counts: {0: 40, 1: 40}

=== GSE168227 ===
Expression: /content/drive/MyDrive/oscc_mirna/data/GSE168227_expr_miRNA_zscored.tsv
Metadata  : /content/drive/MyDrive/oscc_mirna/data/GSE168227_sample_metadata_fixed.tsv
  Merged shape: (48, 407)
  Label value counts: {1: 30, 0: 18}

=== GSE113956 ===
Expression: /content/drive/MyDrive/oscc_mirna/data/GSE113956_expr_miRNA_zscored.tsv
Metadata  : /content/drive/MyDrive/oscc_mirna/data/GSE113956_sample_met

In [14]:
# Build pooled GEO dataset and identify miRNA expression columns

# 1) Pool all GEO datasets together
geo_all = pd.concat(geo_tables.values(), ignore_index=True)

# Keep only rows with a valid diagnostic label
geo_all = geo_all.dropna(subset=[DIAG_LABEL_COL]).copy()
geo_all[DIAG_LABEL_COL] = geo_all[DIAG_LABEL_COL].astype(int)

print("Pooled GEO shape:", geo_all.shape)
print("Pooled label distribution:", geo_all[DIAG_LABEL_COL].value_counts().to_dict())

# 2) Identify miRNA columns by name pattern
expr_cols_geo = [
    c for c in geo_all.columns
    if c.lower().startswith('hsa-') or
       c.lower().startswith('hsa_mir') or
       c.lower().startswith('mir-')
]

print("Number of miRNA columns:", len(expr_cols_geo))
print("First 10 miRNA columns:", expr_cols_geo[:10])


Pooled GEO shape: (269, 36362)
Pooled label distribution: {1: 164, 0: 105}
Number of miRNA columns: 5128
First 10 miRNA columns: ['hsa-let-7a', 'hsa-let-7a*', 'hsa-let-7b', 'hsa-let-7b*', 'hsa-let-7c', 'hsa-let-7c*', 'hsa-let-7d', 'hsa-let-7d*', 'hsa-let-7e', 'hsa-let-7e*']


In [15]:
# Univariate AUC for each miRNA: OSCC (1) vs normal (0) on pooled GEO data

y_diag = geo_all[DIAG_LABEL_COL].values

univ_diag_results = []

for mir in expr_cols_geo:
    x = geo_all[mir].values

    # Skip constant features
    if np.nanstd(x) == 0:
        continue

    try:
        auc = roc_auc_score(y_diag, x)
    except ValueError:
        # This happens if y has only one class for some reason; just skip
        continue

    # Flip direction if AUC < 0.5 so that higher values mean higher OSCC risk
    if auc < 0.5:
        auc = 1.0 - auc

    univ_diag_results.append({'miRNA': mir, 'AUC': auc})

univ_diag_df = (
    pd.DataFrame(univ_diag_results)
    .sort_values('AUC', ascending=False)
    .reset_index(drop=True)
)

print("Number of miRNAs tested:", len(univ_diag_df))
univ_diag_df.head(20)


Number of miRNAs tested: 50


,miRNA,AUC
0,hsa-let-7c,0.832695
1,hsa-miR-617,0.810656
2,hsa-miR-567,0.754297
3,hsa-miR-542-5p,0.749129
4,hsa-miR-636,0.733537
5,hsa-miR-206,0.694628
6,hsa-miR-142-5p,0.686672
7,hsa-miR-1,0.679588
8,hsa-miR-484,0.664837
9,hsa-miR-299-5p,0.650784


from matplotlib import pyplot as plt
_df_0['AUC'].plot(kind='hist', bins=20, title='AUC')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['AUC'].plot(kind='line', figsize=(8, 4), title='AUC')
plt.gca().spines[['top', 'right']].set_visible(False)

In [16]:
# Select top K miRNAs by univariate AUC as candidates for panel search

K_DIAG = 20   # You can change this later (e.g., 15–30)

candidate_mirnas_diag = univ_diag_df.head(K_DIAG)['miRNA'].tolist()

print("K_DIAG =", K_DIAG)
print("Number of candidate miRNAs:", len(candidate_mirnas_diag))
print("Candidate miRNAs:")
print(candidate_mirnas_diag)


K_DIAG = 20
Number of candidate miRNAs: 20
Candidate miRNAs:
['hsa-let-7c', 'hsa-miR-617', 'hsa-miR-567', 'hsa-miR-542-5p', 'hsa-miR-636', 'hsa-miR-206', 'hsa-miR-142-5p', 'hsa-miR-1', 'hsa-miR-484', 'hsa-miR-299-5p', 'hsa-miR-648', 'hsa-miR-650', 'hsa-miR-133b', 'hsa-miR-564', 'hsa-miR-630', 'hsa-miR-107', 'hsa-miR-409-3p', 'hsa-miR-498', 'hsa-miR-142-3p', 'hsa-miR-198']


In [17]:
from itertools import combinations

def evaluate_panel(miRNA_list, X_df, y, n_splits=5, random_state=RANDOM_SEED):
    """
    Evaluate a given miRNA panel using stratified k-fold CV with logistic regression.
    Returns: (mean_auc, std_auc)
    """
    X = X_df[miRNA_list].values
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    aucs = []

    for train_idx, test_idx in skf.split(X, y):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Standardize within each fold
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        model = LogisticRegression(
            penalty='l2',
            solver='lbfgs',
            max_iter=5000
        )
        model.fit(X_train, y_train)
        y_prob = model.predict_proba(X_test)[:, 1]

        auc = roc_auc_score(y_test, y_prob)
        aucs.append(auc)

    return float(np.mean(aucs)), float(np.std(aucs))

print("evaluate_panel() defined.")


evaluate_panel() defined.


In [18]:
# Panel search for diagnostic model: panels of size 1 to MAX_PANEL_SIZE

MAX_PANEL_SIZE = 5   # you can later change to 3, 4, or 5 if needed

X_all_diag = geo_all[expr_cols_geo]
y_diag = geo_all[DIAG_LABEL_COL].values

panel_results = []

for k in range(1, MAX_PANEL_SIZE + 1):
    print(f"Evaluating panels of size {k} ...")
    for combo in combinations(candidate_mirnas_diag, k):
        combo = list(combo)
        mean_auc, std_auc = evaluate_panel(combo, X_all_diag, y_diag)
        panel_results.append({
            'size': k,
            'miRNAs': combo,
            'mean_auc': mean_auc,
            'std_auc': std_auc,
        })

# Put results into a DataFrame and show the top 10 panels overall
panel_df = pd.DataFrame(panel_results).sort_values('mean_auc', ascending=False).reset_index(drop=True)
panel_df.head(10)


Evaluating panels of size 1 ...
Evaluating panels of size 2 ...
Evaluating panels of size 3 ...
Evaluating panels of size 4 ...
Evaluating panels of size 5 ...


,size,miRNAs,mean_auc,std_auc
0,5,"[hsa-let-7c, hsa-miR-617, hsa-miR-542-5p, hsa-...",0.922682,0.032185
1,5,"[hsa-let-7c, hsa-miR-617, hsa-miR-542-5p, hsa-...",0.922276,0.024410
2,5,"[hsa-let-7c, hsa-miR-617, hsa-miR-542-5p, hsa-...",0.920554,0.025319
3,5,"[hsa-let-7c, hsa-miR-617, hsa-miR-542-5p, hsa-...",0.919679,0.018162
4,5,"[hsa-let-7c, hsa-miR-617, hsa-miR-636, hsa-miR...",0.919183,0.033870
5,5,"[hsa-let-7c, hsa-miR-617, hsa-miR-542-5p, hsa-...",0.919102,0.025737
6,4,"[hsa-let-7c, hsa-miR-617, hsa-miR-636, hsa-miR...",0.918867,0.031365
7,5,"[hsa-let-7c, hsa-miR-617, hsa-miR-542-5p, hsa-...",0.918488,0.027576
8,5,"[hsa-let-7c, hsa-miR-617, hsa-miR-542-5p, hsa-...",0.918263,0.026658
9,5,"[hsa-let-7c, hsa-miR-617, hsa-miR-542-5p, hsa-...",0.918263,0.033684


In [19]:
# Best diagnostic panel for each panel size (1..MAX_PANEL_SIZE)

best_by_size = (
    panel_df
    .sort_values('mean_auc', ascending=False)
    .groupby('size')
    .head(1)
    .reset_index(drop=True)
)

best_by_size


,size,miRNAs,mean_auc,std_auc
0,5,"[hsa-let-7c, hsa-miR-617, hsa-miR-542-5p, hsa-...",0.922682,0.032185
1,4,"[hsa-let-7c, hsa-miR-617, hsa-miR-636, hsa-miR...",0.918867,0.031365
2,3,"[hsa-let-7c, hsa-miR-617, hsa-miR-542-5p]",0.910381,0.037007
3,2,"[hsa-let-7c, hsa-miR-617]",0.902724,0.036933
4,1,[hsa-let-7c],0.827895,0.036750


In [22]:
from sklearn.metrics import confusion_matrix

def evaluate_panel_metrics(miRNA_list, X_df, y, n_splits=5, threshold=0.5, random_state=RANDOM_SEED):
    """
    Evaluate a miRNA panel with stratified k-fold CV.
    Returns:
        mean_auc, mean_sensitivity, mean_specificity
    """
    X = X_df[miRNA_list].values
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    aucs = []
    sens_list = []
    spec_list = []

    for train_idx, test_idx in skf.split(X, y):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[train_idx]  # y[train_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Standardize within each fold
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        model = LogisticRegression(
            penalty='l2',
            solver='lbfgs',
            max_iter=5000
        )
        model.fit(X_train, y_train)
        y_prob = model.predict_proba(X_test)[:, 1]

        # AUC
        auc = roc_auc_score(y_test, y_prob)
        aucs.append(auc)

        # Convert probabilities to binary predictions at given threshold
        y_pred = (y_prob >= threshold).astype(int)

        # Confusion matrix: labels [0,1] = [normal, OSCC]
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred, labels=[0, 1]).ravel()

        # Sensitivity = TP / (TP + FN)
        sens = tp / (tp + fn) if (tp + fn) > 0 else np.nan
        # Specificity = TN / (TN + FP)
        spec = tn / (tn + fp) if (tn + fp) > 0 else np.nan

        sens_list.append(sens)
        spec_list.append(spec)

    mean_auc = float(np.nanmean(aucs))
    mean_sens = float(np.nanmean(sens_list))
    mean_spec = float(np.nanmean(spec_list))

    return mean_auc, mean_sens, mean_spec

print("evaluate_panel_metrics() defined.")


evaluate_panel_metrics() defined.


In [23]:
# <<< CELL: define 20-miRNA expanded panel >>>

panel_20 = [
    # TODO: paste your 20 miRNA names here as strings, e.g.:
    # 'hsa-miR-375',
    # 'hsa-miR-139-5p',
    # ...
]

# Quick sanity checks
print("Number of miRNAs in panel_20:", len(panel_20))
missing = [m for m in panel_20 if m not in geo_all.columns]
print("Missing miRNAs in geo_all (should be []):", missing)


Number of miRNAs in panel_20: 0
Missing miRNAs in geo_all (should be []): []


In [29]:
from sklearn.linear_model import LogisticRegressionCV

def fit_logistic_panel(
    panel_miRNAs,
    X_df,
    y,
    use_elastic_net=True,
    l1_ratio=0.5,
    n_splits=5
):
    """
    Fit logistic regression on given miRNA panel.
    If use_elastic_net=True, uses elastic-net with CV to tune C.
    Returns: model, scaler, y_prob_full, cv_auc
    """
    X = X_df[panel_miRNAs].values
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    if use_elastic_net:
        # Elastic-net LR with cross-validated C (like your earlier run)
        model = LogisticRegressionCV(
            Cs=10,
            cv=skf,
            penalty='elasticnet',
            solver='saga',
            l1_ratios=[l1_ratio],
            scoring='roc_auc',
            max_iter=5000,
            n_jobs=-1,
            refit=True
        )
    else:
        # Plain L2 logistic regression
        model = LogisticRegression(
            penalty='l2',
            solver='lbfgs',
            max_iter=5000
        )

    model.fit(X_scaled, y)

    # Cross-validated AUC is available in model.scores_ if using LogisticRegressionCV
    if use_elastic_net:
        # model.scores_ is dict: {class_label: array(cv_folds x Cs)}
        auc_scores = model.scores_[1]  # class 1 = OSCC
        cv_auc = float(auc_scores.mean())
    else:
        cv_auc = np.nan  # not using CV here

    # Predicted probabilities on full data
    y_prob_full = model.predict_proba(X_scaled)[:, 1]

    return model, scaler, y_prob_full, cv_auc



In [30]:
y_diag = geo_all[DIAG_LABEL_COL].values
X_all_diag = geo_all[expr_cols_geo]

model_20, scaler_20, y_prob_20, cv_auc_20 = fit_logistic_panel(
    panel_20,
    X_all_diag,
    y_diag,
    use_elastic_net=True,
    l1_ratio=0.5,
    n_splits=5
)

print("20-miRNA panel – CV AUC:", cv_auc_20)


ValueError: Found array with 0 feature(s) (shape=(269, 0)) while a minimum of 1 is required by StandardScaler.

In [31]:
print("panel_20 contents:")
print(panel_20)
print("\nNumber of miRNAs in panel_20:", len(panel_20))

print("\nHow many of these exist as columns in geo_all?")
present = [m for m in panel_20 if m in geo_all.columns]
missing = [m for m in panel_20 if m not in geo_all.columns]

print("  Present:", len(present))
print("  Missing:", len(missing))
print("\nMissing names (first 20):", missing[:20])


panel_20 contents:
[]

Number of miRNAs in panel_20: 0

How many of these exist as columns in geo_all?
  Present: 0
  Missing: 0

Missing names (first 20): []


In [32]:
from sklearn.linear_model import LogisticRegressionCV

# Use a reasonably large pool of strong miRNAs, e.g. top 74 by univariate AUC
M = 74   # you can adjust (e.g. 50–100)

top_m_mirnas = univ_diag_df.head(M)['miRNA'].tolist()
print("Number of top-m miRNAs:", len(top_m_mirnas))

X_top_m = geo_all[top_m_mirnas].values
y_diag = geo_all[DIAG_LABEL_COL].values

# Standardize once for the CV fitting
scaler_all = StandardScaler()
X_top_m_scaled = scaler_all.fit_transform(X_top_m)

# Elastic-net logistic regression with CV on C
elastic_cv = LogisticRegressionCV(
    Cs=10,
    cv=5,
    penalty='elasticnet',
    solver='saga',
    l1_ratios=[0.5],          # 0.5 = balanced l1/l2, adjust if needed
    scoring='roc_auc',
    max_iter=5000,
    n_jobs=-1,
    refit=True,
    random_state=RANDOM_SEED
)

elastic_cv.fit(X_top_m_scaled, y_diag)

# Cross-validated AUC for this model
auc_scores = elastic_cv.scores_[1]   # scores_ keyed by class label (1 = OSCC)
cv_auc_elastic = float(auc_scores.mean())
print("Elastic-net model CV AUC:", cv_auc_elastic)


Number of top-m miRNAs: 50
Elastic-net model CV AUC: 0.7331231962481962


In [33]:
# Coefficients for the elastic-net model
coef_all = elastic_cv.coef_[0]   # shape (M,)
coef_table_all = pd.DataFrame({
    'miRNA': top_m_mirnas,
    'coef': coef_all,
    'abs_coef': np.abs(coef_all),
})

# Keep only miRNAs with non-zero coefficients
nonzero = coef_table_all[coef_table_all['coef'] != 0].copy()
nonzero_sorted = nonzero.sort_values('abs_coef', ascending=False).reset_index(drop=True)

print("Number of miRNAs with non-zero coef:", nonzero_sorted.shape[0])
display(nonzero_sorted.head(25))


Number of miRNAs with non-zero coef: 13


,miRNA,coef,abs_coef
0,hsa-let-7c,-0.581487,0.581487
1,hsa-miR-617,-0.578714,0.578714
2,hsa-miR-542-5p,0.336158,0.336158
3,hsa-miR-636,0.281682,0.281682
4,hsa-miR-567,-0.164174,0.164174
5,hsa-miR-299-5p,-0.147393,0.147393
6,hsa-miR-409-3p,-0.082609,0.082609
7,hsa-miR-206,-0.068630,0.068630
8,hsa-miR-484,0.050073,0.050073
9,hsa-miR-1,-0.049651,0.049651


In [35]:
sizes = range(5, 21)  # 5 to 20 miRNAs
results_by_size = []

for k in sizes:
    panel_k = nonzero_sorted.head(k)['miRNA'].tolist()
    auc, sens, spec = evaluate_panel_metrics(panel_k, X_all_diag, y_diag, n_splits=5, threshold=0.5)
    results_by_size.append({'size': k, 'AUC': auc, 'sens': sens, 'spec': spec})

results_by_size_df = pd.DataFrame(results_by_size)
display(results_by_size_df)


,size,AUC,sens,spec
0,5,0.908207,0.871780,0.800000
1,6,0.918110,0.889962,0.800000
2,7,0.916640,0.883902,0.800000
3,8,0.914890,0.896023,0.800000
4,9,0.917244,0.902083,0.809524
5,10,0.916351,0.902083,0.800000
6,11,0.914899,0.901894,0.800000
7,12,0.913420,0.901894,0.800000
8,13,0.909091,0.901894,0.800000
9,14,0.909091,0.901894,0.800000


In [37]:
# Derive panels from the elastic-net model coefficients

# coef_all and top_m_mirnas should come from the elastic-net fit (elastic_cv)
coef_all = elastic_cv.coef_[0]   # shape = (number of top_m_mirnas,)
coef_table_all = pd.DataFrame({
    'miRNA': top_m_mirnas,
    'coef': coef_all,
    'abs_coef': np.abs(coef_all),
})

# Keep only miRNAs with non-zero coefficients and rank by |coef|
nonzero_sorted = (
    coef_table_all[coef_table_all['coef'] != 0]
    .sort_values('abs_coef', ascending=False)
    .reset_index(drop=True)
)

print("Number of miRNAs with non-zero coef:", nonzero_sorted.shape[0])
display(nonzero_sorted.head(20))

# Define nested panels by size (20, 13, 10, 8)
panel_20 = nonzero_sorted.head(20)['miRNA'].tolist()
panel_13 = nonzero_sorted.head(13)['miRNA'].tolist()
panel_10 = nonzero_sorted.head(10)['miRNA'].tolist()
panel_8  = nonzero_sorted.head(8)['miRNA'].tolist()

print("\n20-miRNA panel:", panel_20)
print("13-miRNA panel:", panel_13)
print("10-miRNA panel:", panel_10)
print("8-miRNA panel :", panel_8)


Number of miRNAs with non-zero coef: 13


,miRNA,coef,abs_coef
0,hsa-let-7c,-0.581487,0.581487
1,hsa-miR-617,-0.578714,0.578714
2,hsa-miR-542-5p,0.336158,0.336158
3,hsa-miR-636,0.281682,0.281682
4,hsa-miR-567,-0.164174,0.164174
5,hsa-miR-299-5p,-0.147393,0.147393
6,hsa-miR-409-3p,-0.082609,0.082609
7,hsa-miR-206,-0.068630,0.068630
8,hsa-miR-484,0.050073,0.050073
9,hsa-miR-1,-0.049651,0.049651



20-miRNA panel: ['hsa-let-7c', 'hsa-miR-617', 'hsa-miR-542-5p', 'hsa-miR-636', 'hsa-miR-567', 'hsa-miR-299-5p', 'hsa-miR-409-3p', 'hsa-miR-206', 'hsa-miR-484', 'hsa-miR-1', 'hsa-miR-650', 'hsa-miR-142-5p', 'hsa-miR-498']
13-miRNA panel: ['hsa-let-7c', 'hsa-miR-617', 'hsa-miR-542-5p', 'hsa-miR-636', 'hsa-miR-567', 'hsa-miR-299-5p', 'hsa-miR-409-3p', 'hsa-miR-206', 'hsa-miR-484', 'hsa-miR-1', 'hsa-miR-650', 'hsa-miR-142-5p', 'hsa-miR-498']
10-miRNA panel: ['hsa-let-7c', 'hsa-miR-617', 'hsa-miR-542-5p', 'hsa-miR-636', 'hsa-miR-567', 'hsa-miR-299-5p', 'hsa-miR-409-3p', 'hsa-miR-206', 'hsa-miR-484', 'hsa-miR-1']
8-miRNA panel : ['hsa-let-7c', 'hsa-miR-617', 'hsa-miR-542-5p', 'hsa-miR-636', 'hsa-miR-567', 'hsa-miR-299-5p', 'hsa-miR-409-3p', 'hsa-miR-206']


In [38]:
X_all_diag = geo_all[expr_cols_geo]
y_diag = geo_all[DIAG_LABEL_COL].values

for name, panel in [
    ("20-miRNA", panel_20),
    ("13-miRNA", panel_13),
    ("10-miRNA", panel_10),
    ("8-miRNA",  panel_8),
]:
    auc, sens, spec = evaluate_panel_metrics(panel, X_all_diag, y_diag, n_splits=5, threshold=0.5)
    print(f"{name:8s} | AUC = {auc:.3f}, Sens = {sens:.3f}, Spec = {spec:.3f}")


20-miRNA | AUC = 0.909, Sens = 0.902, Spec = 0.800
13-miRNA | AUC = 0.909, Sens = 0.902, Spec = 0.800
10-miRNA | AUC = 0.916, Sens = 0.902, Spec = 0.800
8-miRNA  | AUC = 0.915, Sens = 0.896, Spec = 0.800


In [39]:
def fit_plain_logistic(panel_miRNAs, X_df, y):
    X = X_df[panel_miRNAs].values
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    model = LogisticRegression(
        penalty='l2',
        solver='lbfgs',
        max_iter=5000
    )
    model.fit(X_scaled, y)
    y_prob = model.predict_proba(X_scaled)[:, 1]
    return model, scaler, y_prob

model_10, scaler_10, y_prob_10 = fit_plain_logistic(panel_10, X_all_diag, y_diag)


In [40]:
from sklearn.metrics import roc_curve

def compute_thresholds(y_true, y_prob, target_sens=0.95, target_spec=0.95):
    fpr, tpr, thresholds = roc_curve(y_true, y_prob)  # thresholds from high→low

    sens_arr = tpr
    spec_arr = 1 - fpr

    # High-sensitivity threshold (T_low): smallest threshold with sens >= target_sens
    mask_low = sens_arr >= target_sens
    if mask_low.any():
        idx_low = np.where(mask_low)[0][0]
        T_low = thresholds[idx_low]
        sens_low = sens_arr[idx_low]
        spec_low = spec_arr[idx_low]
    else:
        T_low = sens_low = spec_low = None

    # High-specificity threshold (T_high): largest threshold with spec >= target_spec
    mask_high = spec_arr >= target_spec
    if mask_high.any():
        idx_high = np.where(mask_high)[0][-1]
        T_high = thresholds[idx_high]
        sens_high = sens_arr[idx_high]
        spec_high = spec_arr[idx_high]
    else:
        T_high = sens_high = spec_high = None

    # Balanced threshold (T_mod): maximise Youden index
    J = sens_arr + spec_arr - 1
    idx_mod = np.argmax(J)
    T_mod = thresholds[idx_mod]
    sens_mod = sens_arr[idx_mod]
    spec_mod = spec_arr[idx_mod]

    return {
        'T_low': T_low,
        'sens_at_T_low': sens_low,
        'spec_at_T_low': spec_low,
        'T_high': T_high,
        'sens_at_T_high': sens_high,
        'spec_at_T_high': spec_high,
        'T_mod': T_mod,
        'sens_at_T_mod': sens_mod,
        'spec_at_T_mod': spec_mod,
    }

thresholds_10 = compute_thresholds(
    y_true=y_diag,
    y_prob=y_prob_10,
    target_sens=0.95,   # for high-sensitivity point
    target_spec=0.95    # or 0.97 if you want stricter high-specificity
)

thresholds_10


{'T_low': np.float64(0.3765508101087591),
 'sens_at_T_low': np.float64(0.9512195121951219),
 'spec_at_T_low': np.float64(0.7428571428571429),
 'T_high': np.float64(0.8497519029611463),
 'sens_at_T_high': np.float64(0.6585365853658537),
 'spec_at_T_high': np.float64(0.9523809523809523),
 'T_mod': np.float64(0.5306405126854955),
 'sens_at_T_mod': np.float64(0.9207317073170732),
 'spec_at_T_mod': np.float64(0.819047619047619)}

In [41]:
panels = {
    "20-miRNA": panel_20,
    "13-miRNA": panel_13,
    "10-miRNA": panel_10,
    "8-miRNA":  panel_8,
}

summary_rows = []

for name, panel in panels.items():
    # Fit model on full data for this panel
    model, scaler, y_prob = fit_plain_logistic(panel, X_all_diag, y_diag)

    # Get thresholds (you can change target_spec to 0.97 if you want stricter rule-in)
    info = compute_thresholds(
        y_true=y_diag,
        y_prob=y_prob,
        target_sens=0.95,
        target_spec=0.95,
    )

    # Get CV AUC for this panel at any threshold (AUC is threshold-free)
    auc, _, _ = evaluate_panel_metrics(panel, X_all_diag, y_diag, n_splits=5, threshold=0.5)

    summary_rows.append({
        "panel": name,
        "size": len(panel),
        "cv_auc": auc,
        "T_low":  info['T_low'],
        "sens_T_low": info['sens_at_T_low'],
        "spec_T_low": info['spec_at_T_low'],
        "T_high": info['T_high'],
        "sens_T_high": info['sens_at_T_high'],
        "spec_T_high": info['spec_at_T_high'],
        "T_mod":  info['T_mod'],
        "sens_T_mod": info['sens_at_T_mod'],
        "spec_T_mod": info['spec_at_T_mod'],
    })

summary_df = pd.DataFrame(summary_rows)
summary_df


,panel,size,cv_auc,T_low,sens_T_low,spec_T_low,T_high,sens_T_high,spec_T_high,T_mod,sens_T_mod,spec_T_mod
0,20-miRNA,13,0.909091,0.426458,0.951220,0.771429,0.886663,0.603659,0.952381,0.527311,0.932927,0.838095
1,13-miRNA,13,0.909091,0.426458,0.951220,0.771429,0.886663,0.603659,0.952381,0.527311,0.932927,0.838095
2,10-miRNA,10,0.916351,0.376551,0.951220,0.742857,0.849752,0.658537,0.952381,0.530641,0.920732,0.819048
3,8-miRNA,8,0.914890,0.362560,0.957317,0.742857,0.837758,0.676829,0.952381,0.511057,0.926829,0.809524


In [42]:
# Choose which panel to use for sensitivity/specificity analysis
# You can switch this to panel_10 if that's your main clinical panel
BASE_PANEL = panel_20   # or: panel_10

print("Base panel size:", len(BASE_PANEL))
print("Base panel miRNAs:", BASE_PANEL)


Base panel size: 13
Base panel miRNAs: ['hsa-let-7c', 'hsa-miR-617', 'hsa-miR-542-5p', 'hsa-miR-636', 'hsa-miR-567', 'hsa-miR-299-5p', 'hsa-miR-409-3p', 'hsa-miR-206', 'hsa-miR-484', 'hsa-miR-1', 'hsa-miR-650', 'hsa-miR-142-5p', 'hsa-miR-498']


In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

def fit_weighted_logistic(panel_miRNAs, X_df, y, class_weight=None):
    """
    Fit a penalized logistic regression on selected miRNAs with optional class weights.
    Returns: model, scaler
    """
    X = X_df[panel_miRNAs].values
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    model = LogisticRegression(
        penalty='l2',
        solver='lbfgs',
        max_iter=5000,
        class_weight=class_weight
    )
    model.fit(X_scaled, y)
    return model, scaler


In [44]:
# High-sensitivity model: weight OSCC (1) more to reduce false negatives
sens_class_weight = {0: 1.0, 1: 3.0}
model_sens, scaler_sens = fit_weighted_logistic(
    BASE_PANEL, X_all_diag, y_diag, class_weight=sens_class_weight
)

# High-specificity model: weight normals (0) more to reduce false positives
spec_class_weight = {0: 3.0, 1: 1.0}
model_spec, scaler_spec = fit_weighted_logistic(
    BASE_PANEL, X_all_diag, y_diag, class_weight=spec_class_weight
)


In [45]:
# Coefficients from each model
coef_sens = model_sens.coef_[0]
coef_spec = model_spec.coef_[0]

df_sens = pd.DataFrame({
    'miRNA': BASE_PANEL,
    'coef_sens': coef_sens,
    'abs_coef_sens': np.abs(coef_sens),
}).sort_values('abs_coef_sens', ascending=False).reset_index(drop=True)

df_spec = pd.DataFrame({
    'miRNA': BASE_PANEL,
    'coef_spec': coef_spec,
    'abs_coef_spec': np.abs(coef_spec),
}).sort_values('abs_coef_spec', ascending=False).reset_index(drop=True)

print("Top sensitivity-weighted miRNAs:")
display(df_sens.head(10))

print("Top specificity-weighted miRNAs:")
display(df_spec.head(10))


Top sensitivity-weighted miRNAs:


,miRNA,coef_sens,abs_coef_sens
0,hsa-miR-617,-1.352316,1.352316
1,hsa-miR-636,0.971876,0.971876
2,hsa-miR-542-5p,0.918967,0.918967
3,hsa-let-7c,-0.874595,0.874595
4,hsa-miR-409-3p,-0.610293,0.610293
5,hsa-miR-299-5p,-0.529211,0.529211
6,hsa-miR-498,0.435701,0.435701
7,hsa-miR-567,-0.324013,0.324013
8,hsa-miR-484,0.255493,0.255493
9,hsa-miR-142-5p,0.204629,0.204629


Top specificity-weighted miRNAs:


,miRNA,coef_spec,abs_coef_spec
0,hsa-miR-617,-1.392411,1.392411
1,hsa-let-7c,-0.816893,0.816893
2,hsa-miR-542-5p,0.674240,0.674240
3,hsa-miR-636,0.628193,0.628193
4,hsa-miR-409-3p,-0.584284,0.584284
5,hsa-miR-650,0.352251,0.352251
6,hsa-miR-299-5p,-0.320184,0.320184
7,hsa-miR-567,-0.292799,0.292799
8,hsa-miR-484,0.259073,0.259073
9,hsa-miR-1,-0.154801,0.154801


In [46]:
TOP_SENS = 8
TOP_SPEC = 8

sens_markers = df_sens.head(TOP_SENS)['miRNA'].tolist()
spec_markers = df_spec.head(TOP_SPEC)['miRNA'].tolist()

sens_set = set(sens_markers)
spec_set = set(spec_markers)

both = sorted(list(sens_set & spec_set))
sens_only = sorted(list(sens_set - spec_set))
spec_only = sorted(list(spec_set - sens_set))

marker_roles = []

for m in sens_only:
    marker_roles.append({'miRNA': m, 'role': 'sensitivity-dominant'})
for m in spec_only:
    marker_roles.append({'miRNA': m, 'role': 'specificity-dominant'})
for m in both:
    marker_roles.append({'miRNA': m, 'role': 'shared'})

marker_roles_df = pd.DataFrame(marker_roles).sort_values('role')
marker_roles_df


,miRNA,role
0,hsa-miR-498,sensitivity-dominant
2,hsa-let-7c,shared
3,hsa-miR-299-5p,shared
4,hsa-miR-409-3p,shared
5,hsa-miR-542-5p,shared
6,hsa-miR-567,shared
7,hsa-miR-617,shared
8,hsa-miR-636,shared
1,hsa-miR-650,specificity-dominant


In [47]:
marker_roles_df.to_csv('table_mirna_roles_sens_vs_spec.csv', index=False)


In [48]:
# Join roles with sensitivity- and specificity-model coefficients

roles = marker_roles_df.copy()   # miRNA, role

# Make dataframes keyed by miRNA
df_sens_key = df_sens[['miRNA', 'coef_sens', 'abs_coef_sens']]
df_spec_key = df_spec[['miRNA', 'coef_spec', 'abs_coef_spec']]

roles_full = (
    roles
    .merge(df_sens_key, on='miRNA', how='left')
    .merge(df_spec_key, on='miRNA', how='left')
    .sort_values('role')
    .reset_index(drop=True)
)

display(roles_full)

# Save for manuscript
roles_full.to_csv('table_mirna_roles_sens_spec_with_coefs.csv', index=False)
print("Saved: table_mirna_roles_sens_spec_with_coefs.csv")


,miRNA,role,coef_sens,abs_coef_sens,coef_spec,abs_coef_spec
0,hsa-miR-498,sensitivity-dominant,0.435701,0.435701,0.145832,0.145832
1,hsa-let-7c,shared,-0.874595,0.874595,-0.816893,0.816893
2,hsa-miR-299-5p,shared,-0.529211,0.529211,-0.320184,0.320184
3,hsa-miR-409-3p,shared,-0.610293,0.610293,-0.584284,0.584284
4,hsa-miR-542-5p,shared,0.918967,0.918967,0.674240,0.674240
5,hsa-miR-567,shared,-0.324013,0.324013,-0.292799,0.292799
6,hsa-miR-617,shared,-1.352316,1.352316,-1.392411,1.392411
7,hsa-miR-636,shared,0.971876,0.971876,0.628193,0.628193
8,hsa-miR-650,specificity-dominant,0.202436,0.202436,0.352251,0.352251


Saved: table_mirna_roles_sens_spec_with_coefs.csv


In [49]:
# nonzero_sorted came from elastic-net (miRNA, coef, abs_coef)
max_k = min(25, nonzero_sorted.shape[0])  # e.g. up to 25 or however many nonzero features you have
sizes = range(5, max_k + 1)

results = []

for k in sizes:
    panel_k = nonzero_sorted.head(k)['miRNA'].tolist()

    # 1) Cross-validated AUC (threshold-free) + sens/spec at 0.5 just for reference
    auc_cv, sens_cv_05, spec_cv_05 = evaluate_panel_metrics(
        panel_k, X_all_diag, y_diag, n_splits=5, threshold=0.5
    )

    # 2) Fit model on full data for this panel
    model_k, scaler_k, y_prob_k = fit_plain_logistic(panel_k, X_all_diag, y_diag)

    # 3) ROC-based thresholds:
    thr_info = compute_thresholds(
        y_true=y_diag,
        y_prob=y_prob_k,
        target_sens=0.95,   # high-sens target
        target_spec=0.97    # high-spec target like your previous spec≈0.97–1.0
    )

    results.append({
        'size': k,
        'panel': panel_k,
        'cv_auc': auc_cv,
        'sens_0.5': sens_cv_05,
        'spec_0.5': spec_cv_05,
        'T_low': thr_info['T_low'],
        'sens_T_low': thr_info['sens_at_T_low'],
        'spec_T_low': thr_info['spec_at_T_low'],
        'T_high': thr_info['T_high'],
        'sens_T_high': thr_info['sens_at_T_high'],
        'spec_T_high': thr_info['spec_at_T_high'],
        'T_mod': thr_info['T_mod'],
        'sens_T_mod': thr_info['sens_at_T_mod'],
        'spec_T_mod': thr_info['spec_at_T_mod'],
    })

results_df = pd.DataFrame(results).sort_values('size').reset_index(drop=True)
display(results_df)


,size,panel,cv_auc,sens_0.5,spec_0.5,T_low,sens_T_low,spec_T_low,T_high,sens_T_high,spec_T_high,T_mod,sens_T_mod,spec_T_mod
0,5,"[hsa-let-7c, hsa-miR-617, hsa-miR-542-5p, hsa-...",0.908207,0.871780,0.800000,0.305120,0.951220,0.647619,0.948958,0.414634,0.971429,0.625757,0.835366,0.895238
1,6,"[hsa-let-7c, hsa-miR-617, hsa-miR-542-5p, hsa-...",0.918110,0.889962,0.800000,0.371004,0.957317,0.723810,0.968154,0.304878,0.971429,0.628186,0.865854,0.895238
2,7,"[hsa-let-7c, hsa-miR-617, hsa-miR-542-5p, hsa-...",0.916640,0.883902,0.800000,0.378073,0.951220,0.742857,0.964659,0.335366,0.971429,0.510584,0.920732,0.809524
3,8,"[hsa-let-7c, hsa-miR-617, hsa-miR-542-5p, hsa-...",0.914890,0.896023,0.800000,0.362560,0.957317,0.742857,0.964279,0.335366,0.971429,0.511057,0.926829,0.809524
4,9,"[hsa-let-7c, hsa-miR-617, hsa-miR-542-5p, hsa-...",0.917244,0.902083,0.809524,0.368946,0.951220,0.742857,0.954206,0.396341,0.971429,0.532243,0.920732,0.828571
5,10,"[hsa-let-7c, hsa-miR-617, hsa-miR-542-5p, hsa-...",0.916351,0.902083,0.800000,0.376551,0.951220,0.742857,0.954429,0.402439,0.971429,0.530641,0.920732,0.819048
6,11,"[hsa-let-7c, hsa-miR-617, hsa-miR-542-5p, hsa-...",0.914899,0.901894,0.800000,0.370740,0.951220,0.733333,0.952949,0.414634,0.971429,0.588868,0.890244,0.876190
7,12,"[hsa-let-7c, hsa-miR-617, hsa-miR-542-5p, hsa-...",0.913420,0.901894,0.800000,0.374419,0.951220,0.733333,0.952324,0.414634,0.971429,0.596690,0.884146,0.876190
8,13,"[hsa-let-7c, hsa-miR-617, hsa-miR-542-5p, hsa-...",0.909091,0.901894,0.800000,0.426458,0.951220,0.771429,0.948781,0.439024,0.971429,0.527311,0.932927,0.838095


In [50]:
# 1) Get reference performance from the largest panel in the sweep
max_size = results_df['size'].max()
ref_row = results_df[results_df['size'] == max_size].iloc[0]
ref_auc = ref_row['cv_auc']

print("Reference (largest) panel size:", max_size)
print("Reference CV AUC:", ref_auc)

# 2) Define acceptable deviations
auc_tolerance = 0.002   # allowed drop from best AUC
target_sens_low = 0.95  # required minimum sensitivity at high-sens point
target_spec_high = 0.97 # required minimum specificity at high-spec point

# 3) Filter for panels that meet all criteria
eligible = results_df[
    (results_df['cv_auc'] >= ref_auc - auc_tolerance) &
    (results_df['sens_T_low']  >= target_sens_low) &
    (results_df['spec_T_high'] >= target_spec_high)
].copy()

print("Number of panels meeting criteria:", eligible.shape[0])

if eligible.shape[0] > 0:
    # Smallest panel size among eligible
    best_size = eligible['size'].min()
    sweet_spot = (
        eligible[eligible['size'] == best_size]
        .sort_values('cv_auc', ascending=False)
        .iloc[0]
    )

    print("\nSweet-spot panel size:", best_size)
    print("CV AUC:", sweet_spot['cv_auc'])
    print("High-sens point:  sens =", sweet_spot['sens_T_low'],
          ", spec =", sweet_spot['spec_T_low'])
    print("High-spec point:  sens =", sweet_spot['sens_T_high'],
          ", spec =", sweet_spot['spec_T_high'])
    print("Balanced point:   sens =", sweet_spot['sens_T_mod'],
          ", spec =", sweet_spot['spec_T_mod'])

    sweet_spot_panel = sweet_spot['panel']  # list of miRNA names
    print("\nSweet-spot miRNAs:")
    print(sweet_spot_panel)
else:
    print("No panel met all criteria; consider relaxing thresholds slightly.")


Reference (largest) panel size: 13
Reference CV AUC: 0.9090909090909092
Number of panels meeting criteria: 9

Sweet-spot panel size: 5
CV AUC: 0.9082070707070707
High-sens point:  sens = 0.9512195121951219 , spec = 0.6476190476190475
High-spec point:  sens = 0.4146341463414634 , spec = 0.9714285714285714
Balanced point:   sens = 0.8353658536585366 , spec = 0.8952380952380953

Sweet-spot miRNAs:
['hsa-let-7c', 'hsa-miR-617', 'hsa-miR-542-5p', 'hsa-miR-636', 'hsa-miR-567']


In [51]:
FINAL_PANEL = sweet_spot_panel
print("Final panel size:", len(FINAL_PANEL))

# CV performance at threshold 0.5 (just to report model behaviour)
auc_cv, sens_cv_05, spec_cv_05 = evaluate_panel_metrics(
    FINAL_PANEL, X_all_diag, y_diag, n_splits=5, threshold=0.5
)
print(f"CV AUC: {auc_cv:.3f}")
print(f"Sens@0.5: {sens_cv_05:.3f}, Spec@0.5: {spec_cv_05:.3f}")

# Fit final model and recompute thresholds on full data
model_final, scaler_final, y_prob_final = fit_plain_logistic(
    FINAL_PANEL, X_all_diag, y_diag
)

thr_final = compute_thresholds(
    y_true=y_diag,
    y_prob=y_prob_final,
    target_sens=0.95,   # high-sensitivity target
    target_spec=0.97    # high-specificity target (same as selection rule)
)
thr_final


Final panel size: 5
CV AUC: 0.908
Sens@0.5: 0.872, Spec@0.5: 0.800


{'T_low': np.float64(0.3051199367099704),
 'sens_at_T_low': np.float64(0.9512195121951219),
 'spec_at_T_low': np.float64(0.6476190476190475),
 'T_high': np.float64(0.9489576269553679),
 'sens_at_T_high': np.float64(0.4146341463414634),
 'spec_at_T_high': np.float64(0.9714285714285714),
 'T_mod': np.float64(0.6257565452077923),
 'sens_at_T_mod': np.float64(0.8353658536585366),
 'spec_at_T_mod': np.float64(0.8952380952380953)}

In [52]:
# --- 1. Make sure both panels exist ---

# Exploratory minimal panel (from results_df selection):
# If you followed the earlier code, this is 'sweet_spot_panel'
exploratory_panel = sweet_spot_panel   # size is probably 5

# 13-miRNA panel: either from your earlier definition or from elastic-net ranking
try:
    panel_13    # check if it's already defined
except NameError:
    # Recreate from nonzero_sorted if needed
    panel_13 = nonzero_sorted.head(13)['miRNA'].tolist()

print("Exploratory panel size:", len(exploratory_panel))
print("13-miRNA panel size   :", len(panel_13))

# --- 2. Helper to compute diagnostics for any panel ---

def panel_diagnostics(name, panel, target_sens=0.95, target_spec=0.97):
    X_all_diag = geo_all[expr_cols_geo]
    y_diag = geo_all[DIAG_LABEL_COL].values

    # Cross-validated AUC (threshold-free)
    auc_cv, _, _ = evaluate_panel_metrics(
        panel,
        X_all_diag,
        y_diag,
        n_splits=5,
        threshold=0.5,   # threshold only used to report sens/spec inside fn, AUC is independent
    )

    # Fit on full data for ROC thresholds
    model, scaler, y_prob = fit_plain_logistic(panel, X_all_diag, y_diag)

    thr = compute_thresholds(
        y_true=y_diag,
        y_prob=y_prob,
        target_sens=target_sens,
        target_spec=target_spec,
    )

    return {
        "panel_name": name,
        "size": len(panel),
        "cv_auc": auc_cv,
        "T_low": thr["T_low"],
        "sens_T_low": thr["sens_at_T_low"],
        "spec_T_low": thr["spec_at_T_low"],
        "T_high": thr["T_high"],
        "sens_T_high": thr["sens_at_T_high"],
        "spec_T_high": thr["spec_at_T_high"],
        "T_mod": thr["T_mod"],
        "sens_T_mod": thr["sens_at_T_mod"],
        "spec_T_mod": thr["spec_at_T_mod"],
    }

# --- 3. Compute metrics for both methods ---

rows = []
rows.append(panel_diagnostics("Exploratory minimal panel", exploratory_panel))
rows.append(panel_diagnostics("Fixed 13-miRNA panel", panel_13))

comparison_df = pd.DataFrame(rows)
display(comparison_df)


Exploratory panel size: 5
13-miRNA panel size   : 13


,panel_name,size,cv_auc,T_low,sens_T_low,spec_T_low,T_high,sens_T_high,spec_T_high,T_mod,sens_T_mod,spec_T_mod
0,Exploratory minimal panel,5,0.908207,0.305120,0.95122,0.647619,0.948958,0.414634,0.971429,0.625757,0.835366,0.895238
1,Fixed 13-miRNA panel,13,0.909091,0.426458,0.95122,0.771429,0.948781,0.439024,0.971429,0.527311,0.932927,0.838095


In [53]:
# roles_full should have: miRNA, role, coef_sens, abs_coef_sens, coef_spec, abs_coef_spec

sens_markers = roles_full[roles_full['role'] == 'sensitivity-dominant']['miRNA'].tolist()
spec_markers = roles_full[roles_full['role'] == 'specificity-dominant']['miRNA'].tolist()
shared_markers = roles_full[roles_full['role'] == 'shared']['miRNA'].tolist()

print("Sensitivity markers:", sens_markers)
print("Specificity markers:", spec_markers)
print("Shared markers     :", shared_markers)


Sensitivity markers: ['hsa-miR-498']
Specificity markers: ['hsa-miR-650']
Shared markers     : ['hsa-let-7c', 'hsa-miR-299-5p', 'hsa-miR-409-3p', 'hsa-miR-542-5p', 'hsa-miR-567', 'hsa-miR-617', 'hsa-miR-636']


In [54]:
X_all_diag = geo_all[expr_cols_geo]
y_diag = geo_all[DIAG_LABEL_COL].values

def fit_panel(panel_mirnas, class_weight):
    X = X_all_diag[panel_mirnas].values
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    model = LogisticRegression(
        penalty='l2',
        solver='lbfgs',
        max_iter=5000,
        class_weight=class_weight
    )
    model.fit(X_scaled, y_diag)
    y_prob = model.predict_proba(X_scaled)[:, 1]
    return model, scaler, y_prob

# Model_SENS: favour catching OSCC (penalise FN)
panel_sens = sorted(list(set(sens_markers + shared_markers)))
model_sens_final, scaler_sens_final, y_prob_sens = fit_panel(
    panel_sens,
    class_weight={0: 1.0, 1: 3.0}   # OSCC 3× weight
)

# Model_SPEC: favour avoiding FP (penalise calling normals as OSCC)
panel_spec = sorted(list(set(spec_markers + shared_markers)))
model_spec_final, scaler_spec_final, y_prob_spec = fit_panel(
    panel_spec,
    class_weight={0: 3.0, 1: 1.0}   # Normal 3× weight
)

print("Panel_SENS miRNAs:", panel_sens)
print("Panel_SPEC miRNAs:", panel_spec)


Panel_SENS miRNAs: ['hsa-let-7c', 'hsa-miR-299-5p', 'hsa-miR-409-3p', 'hsa-miR-498', 'hsa-miR-542-5p', 'hsa-miR-567', 'hsa-miR-617', 'hsa-miR-636']
Panel_SPEC miRNAs: ['hsa-let-7c', 'hsa-miR-299-5p', 'hsa-miR-409-3p', 'hsa-miR-542-5p', 'hsa-miR-567', 'hsa-miR-617', 'hsa-miR-636', 'hsa-miR-650']


In [55]:
# High-sensitivity thresholds for Model_SENS
thr_sens = compute_thresholds(
    y_true=y_diag,
    y_prob=y_prob_sens,
    target_sens=0.98,   # you can set 0.97–0.99
    target_spec=0.0     # don't require spec here
)
thr_sens

# High-specificity thresholds for Model_SPEC
thr_spec = compute_thresholds(
    y_true=y_diag,
    y_prob=y_prob_spec,
    target_sens=0.0,    # don't constrain sens for threshold search
    target_spec=0.97    # or 0.98, depends
)
thr_spec


{'T_low': np.float64(inf),
 'sens_at_T_low': np.float64(0.0),
 'spec_at_T_low': np.float64(1.0),
 'T_high': np.float64(0.8551062844200058),
 'sens_at_T_high': np.float64(0.4268292682926829),
 'spec_at_T_high': np.float64(0.9714285714285714),
 'T_mod': np.float64(0.3644177847665579),
 'sens_at_T_mod': np.float64(0.8414634146341463),
 'spec_at_T_mod': np.float64(0.8761904761904762)}

In [56]:
T_low_sens  = thr_sens['T_low']
T_high_spec = thr_spec['T_high']

def classify_patient(p_sens, p_spec):
    """
    p_sens: prob from sensitivity model
    p_spec: prob from specificity model
    returns: 'low', 'intermediate', or 'high' risk
    """
    if p_spec >= T_high_spec:
        return "high-risk (rule-in OSCC)"
    elif p_sens < T_low_sens:
        return "low-risk (rule-out OSCC)"
    else:
        return "indeterminate"

# Example on the whole dataset:
labels = []
for ps, pp in zip(y_prob_sens, y_prob_spec):
    labels.append(classify_patient(ps, pp))

pd.Series(labels).value_counts(normalize=True)


,proportion
indeterminate,0.442379
low-risk (rule-out OSCC),0.286245
high-risk (rule-in OSCC),0.271375


In [57]:
# assume these already exist from your notebook
# panel_bal : list of miRNAs used for balanced model
# panel_sens, panel_spec : lists for SENS and SPEC models
# model_bal, model_sens_final, model_spec_final : fitted sklearn LogisticRegression
# scaler_bal, scaler_sens_final, scaler_spec_final : StandardScaler objects
# thr_sens, thr_spec, thr_bal : dicts from compute_thresholds()

# 1. Export means/SDs for the final panel (balanced model)
FINAL_MIRNAS = sorted(list(set(panel_bal)))  # union if needed

means = dict(zip(FINAL_MIRNAS, scaler_bal.mean_))
stds  = dict(zip(FINAL_MIRNAS, scaler_bal.scale_))

print("FINAL_MIRNAS =", FINAL_MIRNAS)
print("MEANS_BAL =", means)
print("STDS_BAL  =", stds)

# 2. Export coefficients and intercepts
coef_bal  = dict(zip(FINAL_MIRNAS, model_bal.coef_[0]))
coef_sens = dict(zip(FINAL_MIRNAS, model_sens_final.coef_[0]))
coef_spec = dict(zip(FINAL_MIRNAS, model_spec_final.coef_[0]))

print("INTERCEPT_BAL =", float(model_bal.intercept_[0]))
print("INTERCEPT_SENS =", float(model_sens_final.intercept_[0]))
print("INTERCEPT_SPEC =", float(model_spec_final.intercept_[0]))
print("COEFFS_BAL =", coef_bal)
print("COEFFS_SENS =", coef_sens)
print("COEFFS_SPEC =", coef_spec)

# 3. Export thresholds
print("T_LOW_SENS  =", thr_sens['T_low'])
print("T_HIGH_SPEC =", thr_spec['T_high'])
print("T_MOD_BAL   =", thr_bal['T_mod'])


NameError: name 'panel_bal' is not defined

In [58]:
# 1. Final miRNA list for *all* three models
FINAL_MIRNAS = FINAL_PANEL  # this is the panel you chose as FINAL
print("FINAL_MIRNAS:", FINAL_MIRNAS)

# 2. Means and SDs from the balanced model's scaler
# (order of scaler_final.mean_ matches FINAL_PANEL)
MEANS_BAL = dict(zip(FINAL_PANEL, scaler_final.mean_))
STDS_BAL  = dict(zip(FINAL_PANEL, scaler_final.scale_))

print("MEANS_BAL =", MEANS_BAL)
print("STDS_BAL  =", STDS_BAL)

# 3. Coefficients for the BALANCED model
COEFFS_BAL = dict(zip(FINAL_PANEL, model_final.coef_[0]))
INTERCEPT_BAL = float(model_final.intercept_[0])

print("INTERCEPT_BAL =", INTERCEPT_BAL)
print("COEFFS_BAL =", COEFFS_BAL)

# 4. Coefficients for SENS and SPEC models
#    (these were trained on panel_sens and panel_spec respectively)

COEFFS_SENS_RAW = dict(zip(panel_sens, model_sens_final.coef_[0]))
COEFFS_SPEC_RAW = dict(zip(panel_spec, model_spec_final.coef_[0]))
INTERCEPT_SENS = float(model_sens_final.intercept_[0])
INTERCEPT_SPEC = float(model_spec_final.intercept_[0])

# For the app we want a coefficient for *every* miRNA in FINAL_MIRNAS;
# if a miRNA is not used in SENS or SPEC model, its coef is 0.
COEFFS_SENS = {m: COEFFS_SENS_RAW.get(m, 0.0) for m in FINAL_MIRNAS}
COEFFS_SPEC = {m: COEFFS_SPEC_RAW.get(m, 0.0) for m in FINAL_MIRNAS}

print("INTERCEPT_SENS =", INTERCEPT_SENS)
print("INTERCEPT_SPEC =", INTERCEPT_SPEC)
print("COEFFS_SENS =", COEFFS_SENS)
print("COEFFS_SPEC =", COEFFS_SPEC)

# 5. Thresholds from your ROC analyses
T_LOW_SENS  = thr_sens['T_low']
T_HIGH_SPEC = thr_spec['T_high']
T_MOD_BAL   = thr_final['T_mod']

print("T_LOW_SENS  =", T_LOW_SENS)
print("T_HIGH_SPEC =", T_HIGH_SPEC)
print("T_MOD_BAL   =", T_MOD_BAL)


FINAL_MIRNAS: ['hsa-let-7c', 'hsa-miR-617', 'hsa-miR-542-5p', 'hsa-miR-636', 'hsa-miR-567']
MEANS_BAL = {'hsa-let-7c': np.float64(5.150774478558347e-16), 'hsa-miR-617': np.float64(-7.924268428551303e-16), 'hsa-miR-542-5p': np.float64(5.61302347022384e-16), 'hsa-miR-636': np.float64(-1.2546758345206231e-15), 'hsa-miR-567': np.float64(9.971371105927057e-16)}
STDS_BAL  = {'hsa-let-7c': np.float64(1.0), 'hsa-miR-617': np.float64(1.0), 'hsa-miR-542-5p': np.float64(1.0), 'hsa-miR-636': np.float64(1.0), 'hsa-miR-567': np.float64(1.0)}
INTERCEPT_BAL = 0.8944860036515973
COEFFS_BAL = {'hsa-let-7c': np.float64(-1.1305907135886717), 'hsa-miR-617': np.float64(-1.132775044806725), 'hsa-miR-542-5p': np.float64(0.6441193710722666), 'hsa-miR-636': np.float64(0.7287426710841557), 'hsa-miR-567': np.float64(-0.32865236656758373)}
INTERCEPT_SENS = 2.125602344218705
INTERCEPT_SPEC = -0.19553506646614488
COEFFS_SENS = {'hsa-let-7c': np.float64(-0.9919435707164074), 'hsa-miR-617': np.float64(-1.5179761428621

In [59]:
# FINAL MODEL: 13-miRNA panel from today's elastic-net ranking
FINAL_PANEL = panel_13
print("FINAL_PANEL (13 miRNAs):", FINAL_PANEL)
len(FINAL_PANEL)


FINAL_PANEL (13 miRNAs): ['hsa-let-7c', 'hsa-miR-617', 'hsa-miR-542-5p', 'hsa-miR-636', 'hsa-miR-567', 'hsa-miR-299-5p', 'hsa-miR-409-3p', 'hsa-miR-206', 'hsa-miR-484', 'hsa-miR-1', 'hsa-miR-650', 'hsa-miR-142-5p', 'hsa-miR-498']


13

In [60]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

X_all_diag = geo_all[expr_cols_geo]
y_diag = geo_all[DIAG_LABEL_COL].values

# ---- BALANCED MODEL (no class weights) ----
scaler_final = StandardScaler()
X_bal = scaler_final.fit_transform(X_all_diag[FINAL_PANEL].values)

model_final = LogisticRegression(
    penalty='l2',
    solver='lbfgs',
    max_iter=5000
)
model_final.fit(X_bal, y_diag)
y_prob_final = model_final.predict_proba(X_bal)[:, 1]

# ---- SENSITIVITY-ORIENTED MODEL ----
scaler_sens_final = StandardScaler()
X_sens = scaler_sens_final.fit_transform(X_all_diag[FINAL_PANEL].values)

model_sens_final = LogisticRegression(
    penalty='l2',
    solver='lbfgs',
    max_iter=5000,
    class_weight={0: 1.0, 1: 3.0}  # emphasise OSCC -> high sensitivity
)
model_sens_final.fit(X_sens, y_diag)
y_prob_sens = model_sens_final.predict_proba(X_sens)[:, 1]

# ---- SPECIFICITY-ORIENTED MODEL ----
scaler_spec_final = StandardScaler()
X_spec = scaler_spec_final.fit_transform(X_all_diag[FINAL_PANEL].values)

model_spec_final = LogisticRegression(
    penalty='l2',
    solver='lbfgs',
    max_iter=5000,
    class_weight={0: 3.0, 1: 1.0}  # emphasise normals -> high specificity
)
model_spec_final.fit(X_spec, y_diag)
y_prob_spec = model_spec_final.predict_proba(X_spec)[:, 1]


In [61]:
# thresholds for 13-miRNA panel
thr_final = compute_thresholds(
    y_true=y_diag,
    y_prob=y_prob_final,
    target_sens=0.95,
    target_spec=0.97
)

thr_sens = compute_thresholds(
    y_true=y_diag,
    y_prob=y_prob_sens,
    target_sens=0.95,
    target_spec=0.0
)

thr_spec = compute_thresholds(
    y_true=y_diag,
    y_prob=y_prob_spec,
    target_sens=0.0,
    target_spec=0.97
)

thr_final, thr_sens, thr_spec


({'T_low': np.float64(0.4264578775990947),
  'sens_at_T_low': np.float64(0.9512195121951219),
  'spec_at_T_low': np.float64(0.7714285714285715),
  'T_high': np.float64(0.948781467896416),
  'sens_at_T_high': np.float64(0.43902439024390244),
  'spec_at_T_high': np.float64(0.9714285714285714),
  'T_mod': np.float64(0.5273112073123669),
  'sens_at_T_mod': np.float64(0.9329268292682927),
  'spec_at_T_mod': np.float64(0.8380952380952381)},
 {'T_low': np.float64(0.6540082327784301),
  'sens_at_T_low': np.float64(0.9512195121951219),
  'spec_at_T_low': np.float64(0.780952380952381),
  'T_high': np.float64(0.0004039494668619717),
  'sens_at_T_high': np.float64(1.0),
  'spec_at_T_high': np.float64(0.0),
  'T_mod': np.float64(0.7801584373386374),
  'sens_at_T_mod': np.float64(0.926829268292683),
  'spec_at_T_mod': np.float64(0.8476190476190476)},
 {'T_low': np.float64(inf),
  'sens_at_T_low': np.float64(0.0),
  'spec_at_T_low': np.float64(1.0),
  'T_high': np.float64(0.8042582572876076),
  'sens

In [62]:
FINAL_MIRNAS = FINAL_PANEL
MEANS_BAL = dict(zip(FINAL_PANEL, scaler_final.mean_))
STDS_BAL  = dict(zip(FINAL_PANEL, scaler_final.scale_))
COEFFS_BAL = dict(zip(FINAL_PANEL, model_final.coef_[0]))
INTERCEPT_BAL  = float(model_final.intercept_[0])
COEFFS_SENS = dict(zip(FINAL_PANEL, model_sens_final.coef_[0]))
INTERCEPT_SENS = float(model_sens_final.intercept_[0])
COEFFS_SPEC = dict(zip(FINAL_PANEL, model_spec_final.coef_[0]))
INTERCEPT_SPEC = float(model_spec_final.intercept_[0])

print("FINAL_MIRNAS:", FINAL_MIRNAS)
print("MEANS_BAL =", MEANS_BAL)
print("STDS_BAL  =", STDS_BAL)
print("INTERCEPT_BAL =", INTERCEPT_BAL)
print("INTERCEPT_SENS =", INTERCEPT_SENS)
print("INTERCEPT_SPEC =", INTERCEPT_SPEC)
print("COEFFS_BAL =", COEFFS_BAL)
print("COEFFS_SENS =", COEFFS_SENS)
print("COEFFS_SPEC =", COEFFS_SPEC)


FINAL_MIRNAS: ['hsa-let-7c', 'hsa-miR-617', 'hsa-miR-542-5p', 'hsa-miR-636', 'hsa-miR-567', 'hsa-miR-299-5p', 'hsa-miR-409-3p', 'hsa-miR-206', 'hsa-miR-484', 'hsa-miR-1', 'hsa-miR-650', 'hsa-miR-142-5p', 'hsa-miR-498']
MEANS_BAL = {'hsa-let-7c': np.float64(5.150774478558347e-16), 'hsa-miR-617': np.float64(-7.924268428551303e-16), 'hsa-miR-542-5p': np.float64(5.61302347022384e-16), 'hsa-miR-636': np.float64(-1.2546758345206231e-15), 'hsa-miR-567': np.float64(9.971371105927057e-16), 'hsa-miR-299-5p': np.float64(-2.641422809517101e-17), 'hsa-miR-409-3p': np.float64(1.220007160145711e-15), 'hsa-miR-206': np.float64(-2.0471026773757533e-16), 'hsa-miR-484': np.float64(7.164859370815137e-16), 'hsa-miR-1': np.float64(3.7310097184429054e-16), 'hsa-miR-650': np.float64(3.8465719663592785e-16), 'hsa-miR-142-5p': np.float64(1.8489959666619708e-16), 'hsa-miR-498': np.float64(1.0565691238068404e-16)}
STDS_BAL  = {'hsa-let-7c': np.float64(1.0), 'hsa-miR-617': np.float64(1.0), 'hsa-miR-542-5p': np.flo